In [1]:
import os

import numpy as np

from pyazr import azure2
from lmfit import Parameters, Minimizer

from multiprocess import Pool

from IPython.display import display, update_display

os.environ["OMP_NUM_THREADS"] = "1"

niter  = 10
nprocs = 2

# Define the data labels (in AZURE2 order)
labels = ["Meyer et al. (1976) - 84.3 deg",
          "Meyer et al. (1976) - 114.5 deg",
          "Meyer et al. (1976) - 144.1 deg", 
          "LUNA HPGe (2023)", 
          "LUNA BGO (2023)",
          "Felsenkeller (2023)",
          "ATOMKI (2023)",
          "Notre Dame (2023) - 0 deg",
          "Notre Dame (2023) - 55 deg",
          "Burtebaev et al. (2008)",
          "Lamb et al. (1957)",
          "Bailey et al. (1950)",
          "Vogl et al. (1963)",
          "Rolfs et al. (1974) - 0 deg",
          "Rolfs et al. (1974) - 90 deg"]

# Nuisance parameter map : { index : (value, error) } (in AZURE2 order)
nuisances = { 0 : (1.63, 0.12) }

# Normalization parameter map : { index : (value, error) } (in AZURE2 order)
norms = { 0:  (1, 0.05),   1:  (1, 0.05),   2: (1, 0.05),  
          3:  (1, 0.069),  4:  (1, 0.079),  5: (1, 0.1),  
          6:  (1, 0.1),    7:  (1, 0.1),    8: (1, 0.1),  
          9:  (1, 0.1),    10: (1, 0.0),   11: (1, 0.0), 
          12: (1, 0.0),    13: (1, 0.0),   14: (1, 0.0) }

In [2]:
# We read the .azr file and set the external capture file to speed up the calculation
azr = azure2('12c_pg.azr', nprocs=nprocs)

In [3]:
# We get the initial values from AZURE2
theta0 = azr.params
ntheta = len(theta0)

# Now we add the normalizations
for norm in norms.items( ):
    theta0 = np.concatenate( (theta0, [norm[1][0]]) )

# We'll read the data from the output file since it's already in the center-of-mass frame
y = azr.cross
yerr = azr.cross_err
ndata = sum( len(segment) for segment in y )

In [4]:
# Callback function to print the chi2 at each iteration
def callback(params, iter, resid, *args, **kws):
    if( iter % 10 == 0 ): 
        print( " ", end="\r")
        update_display("Process {} ---- Iteration: {:6d} it Chi2: {:15.4f}".format( args[0], iter, resid ), display_id=str(args[0]))
    pass

# Add nuisance parameter to chi2
def nuisance( theta ):
    nu = 0
    for i in nuisances:
        nu += pow( (theta[i] - nuisances[i][0]) / nuisances[i][1], 2 )
    return nu

# Add nuisance normalization to chi2
def normalization( theta ):
    norm = 0
    for i in norms:
        if( norms[i][1] != 0 ):
            idx = ntheta + i
            norm += pow( (theta[idx] - norms[i][0]) / norms[i][1], 2 )
    return norm

# Calculated squared residuals
def least_squares( mu, theta ):
    res = 0
    for i in range( len( mu ) ):
        idx = ntheta + i
        res += np.sum( pow( (mu[i] - y[i] * theta[idx]) / ( yerr[i] * theta[idx] ), 2 ) )
    return res

#Function to minimize
def func( theta, proc=0 ):
    theta = list( theta.valuesdict().values() )
    mu = azr.calculate( theta[:ntheta], proc=proc )
    fcn = least_squares( mu, theta ) + nuisance( theta ) + normalization( theta )
    return fcn

In [5]:
def fit( args ):
    
    proc, values = args[0], args[1]

    params = Parameters()
    for i in range(len(values)):
        if( i == 0 ): params.add( "param_{}".format(i), value=values[i], vary=True, min=0, max=values[i] * 10 )
        elif( i in [1,5,11] ): params.add( "param_{}".format(i), value=values[i], vary=True, min=values[i] - 0.01, max=values[i] + 0.01 )
        else: params.add( "param_{}".format(i), value=values[i], vary=True, min=-1e8, max=1e8 )

    print( " ", end="\r" )
    mini = Minimizer( func, params, fcn_args=(proc,), iter_cb=callback )
    out = mini.minimize( method="nelder" )
    update_display("Process {} ---- Finished.".format( proc ), display_id=str(proc))
                        
    result = [param.value for _, param in out.params.items()]
    return result

In [6]:
for _ in range( niter ):

    # Sample the initial guesses
    bucket = [ ]
    for _ in range( nprocs ):
        params = np.zeros( len(theta0) )
        for i in range( len(theta0) ):
            if( i not in [1, 5, 11] ): params[i] = np.random.uniform( theta0[i] * 0.8, theta0[i] * 1.2 )
            else: params[i] = theta0[i]
        bucket.append( params )

    for l in range( nprocs ):
        display("Process {} ---- Starting...".format( l ), display_id=str(l))

    with Pool(processes=nprocs) as pool:
        result = pool.map( fit, zip(range( nprocs ), bucket) )

    file = open( "results/initial/samples.txt", "a" )
    for j in range( len( result ) ):
        for k in range( len( result[j] ) ):
            file.write( "{}\t".format(result[j][k]) )
        file.write( "\n" )
    file.close( )

'Process 0 ---- Finished.'

'Process 1 ---- Finished.'

'Process 0 ---- Iteration:   2060 it Chi2:       6536.1433'

'Process 1 ---- Iteration:   2040 it Chi2:       6698.1080'